# Laboratorio 3: Problema de tiempo mínimo y lineal cuadrático

**Nombres:** Sebastián Acuña U. y Patricio Yañez A. <br>
**Fecha:** 1 de octubre de 2024 <br>
**Profesor:** Héctor Ramírez C. <br>
**Auxiliar:** Diego Olguín W. <br>
**Ayudantes:** Carlos Antil C. y Luis Fuentes C. <br>
**Curso:** [MA4703-1] Control Óptimo: Teoría y Laboratorio

In [10]:
# Librerías a utilizar

#using Pkg
#Pkg.add("DifferentialEquations")
#Pkg.add("JuMP")
#Pkg.add("NonlinearSolve")
#Pkg.add("OptimalControl")
#Pkg.add("LaTeXStrings")
#Pkg.add("NLPModelsIpopt")
#Pkg.add("ControlSystems")
#Pkg.add("Interpolations")

using LinearAlgebra
using Plots
using LaTeXStrings
using JuMP: Model as JuMPModel
using Ipopt
using NLPModelsIpopt
using DifferentialEquations
using NonlinearSolve
using OptimalControl: Model as OptimalControlModel
using ControlSystems
using Interpolations

## Parte A: Control de un carro-cohete en tiempo mínimo y método de resolución directo

### Ejercicio 1

Tenemos la ecuación de la dinámica

$$
\ddot{x} = -\alpha x + u
$$

con condiciones de borde $\mathbf{x_0} = (x_0, v_0) = (0,0)$ y $\mathbf{x_{t_f}} = (x_{t_f}, v_{t_f}) = (x_f, 0)$. Nos interesa resolver el problema 

$$
\min_{u \in \mathcal{A}}{t_f}; \ x(t_f;u,\mathbf{x_0}) = x_f; \ v(t_f;u,\mathbf{x_0}) = 0
$$

donde $|u(t)| \leq 1, \; \forall t$. 

Definiendo $v = \dot{x}$ se genera un sistema de ecuaciones cuya forma matricial es

$$
\dot{\begin{pmatrix}
x(t) \\
v(t)
\end{pmatrix}}
=
\begin{pmatrix}
0 & 1 \\
-\alpha & 0
\end{pmatrix}
\begin{pmatrix}
x(t) \\
v(t)
\end{pmatrix}
+
\begin{pmatrix}
0 \\
1
\end{pmatrix}
u(t)
$$

Si discretizamos la dinámica particionando el intervalo $[0,t_f]$ en $N$ puntos $0 = t_1 < t_2 < \cdots < t_{N} = t_f$, donde $t_i = (i-1) \dfrac{t_f}{N-1}$ obtenemos el siguiente sistema 

\begin{align*}
\dfrac{x_{i+1}-x_i}{h} &= v_i \\
\dfrac{v_{i+1}-v_i}{h} &= -\alpha x_i + u_i
\end{align*}

donde $x_i = x(t_i), v_i = v(t_i), u_i = u(t_i)$ y $h = \frac{t_f}{N-1}$. Esto en particular implica que 

$$
\begin{pmatrix}
x_{i+1} \\
v_{i+1}
\end{pmatrix} = \begin{pmatrix}
x_i \\
v_i
\end{pmatrix} + h \begin{pmatrix}
v_i \\
-\alpha x_i
\end{pmatrix} + h \begin{pmatrix}
0 \\
u_i
\end{pmatrix}
$$


In [11]:
# Definición de la discretización de Euler y resolución utilizando JuMP

function euler_discretization(N, α, x_f)
    model = JuMPModel(Ipopt.Optimizer)
    
    @variable(model, 0 <= t_f)
    @variable(model, -1 <= u[1:N-1] <= 1)
    @variable(model, x[1:N])
    @variable(model, v[1:N])
    
    h = t_f / (N - 1)
    
    @constraint(model, x[1] == 0)
    @constraint(model, v[1] == 0)
    @constraint(model, x[N] == x_f)
    @constraint(model, v[N] == 0)
    
    for i in 1:N-1
        @constraint(model, x[i+1] == x[i] + h * v[i])
        @constraint(model, v[i+1] == v[i] + h * (-α * x[i] + u[i]))
    end
    
    @objective(model, Min, t_f)
    
    optimize!(model)
    
    return value.(x), value.(v), value.(u), value(t_f)
end

# Parámetros de juguete
N = 100
α = 1.0
x_f = 10.0

# Resolución
x, v, u, t_f = euler_discretization(N, α, x_f)

println("Tiempo final óptimo: ", t_f)
println("Posiciones: ", x)
println("Velocidades: ", v)
println("Control: ", u)

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      994
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      297

Total number of variables............................:      300
                     variables with only lower bounds:        1
                variables with lower and upper bounds:       99
                     variables with only upper bounds:        0
Total number of equality constraints.................:      202
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999900e-03 1.00e+01 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

### Ejercicio 2

In [10]:
# Distintos valores de N, α y x_f
N = [10^i for i in 2:6]
α = [-4, -1, 0, 2, 50]
x_f = [-7, -3, 0, 10, 500]

# Resolver el problema de tiempo mínimo discretizado probando las combinaciones de N, α, x_f
results = []

for n in N
    for a in α
        for xf in x_f
            try
                x, v, u, t_f = euler_discretization(n, a, xf)
                push!(results, (N=n, α=a, x_f=xf, t_f=t_f))
                println("N: $n, α: $a, x_f: $xf, t_f: $t_f")
            catch e
                println("Error con N: $n, α: $a, x_f: $xf - ", e)
            end
        end
    end
end

results

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:      994
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      297

Total number of variables............................:      300
                     variables with only lower bounds:        1
                variables with lower and upper bounds:       99
                     variables with only upper bounds:        0
Total number of equality constraints.................:      202
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  9.9999900e-03 7.00e+00 0.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

### Ejercicio 3

In [15]:
# Definición y resolución del problema de tiempo mínimo utilizando OptimalControl

function solve_min_time_continuous(α, x_f)
    model = OptimalControlModel(Ipopt.Optimizer, 2, 1)

    @def begin
        t_f ∈ R, variable
        t ∈ [0, t_f], time
        x ∈ R², state
        u ∈ [-1, 1], control
        \dot{x} = [x[2], -α * x[1] + u]
        x(0) = [0, 0]
        x(t_f) = [x_f, 0]
        t_f >= 0
    end

    @objective(model, Min, tf)

    optimize!(model)

    return solution(model)
end

# Parámetros de juguete
α = 1.0
x_f = 10.0

# Resolución
sol = solve_min_time_continuous(α, x_f)

println("Tiempo final óptimo: ", sol.tf)
println("Posiciones: ", sol.x[1,:])
println("Velocidades: ", sol.x[2,:])
println("Control: ", sol.u[1,:])

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///Users/sebastianacunaurzua/Documents/GitHub/Labs-Control/Laboratorio 3/In[15]#11:9\[90mIn[15]:11:9[0;0m]8;;\
        u ∈ [-1, 1], control
        [48;2;120;70;70m\[0;0mdot{x} = [x[2], -α * x[1] + u]
[90m#       ╙ ── [0;0m[91mnot a unary operator[0;0m